# Configuration

In [ ]:
cd '/Users/xusikun/Desktop/Research/[research]tsinghua_options/options-study'

In [2]:
import os 
import scipy.io
import pandas as pd 
import numpy as np

from datetime import datetime

In [3]:
DATA = 'data/'

In [59]:
mat = scipy.io.loadmat(os.path.join(DATA, 'DataFile.mat'))

In [60]:
mat.keys()
print('__header__', mat['__header__'])
print('__version__', mat['__version__'])
print('__globals__', mat['__globals__'])

__header__ b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Fri Jun 11 17:36:09 2021'
__version__ 1.0
__globals__ []


# Contracts

In [92]:
C = mat['Contract'].flatten()[0][0]
dates = [C[idx][0][0] for idx in range(C.shape[0])]
print(C.shape)

(1531, 21)


In [255]:
def get_daily_options_book(arr: np.array, date: str) -> pd.DataFrame:
    book_idx_list = [5, 6, 9, 10, 13, 14, 17, 18]

    # helper functions
    def helper_tabular_option(arr):
        new_arr = np.empty(arr.shape, dtype='object')
        for ridx in range(arr.shape[0]):
            for cidx in range(arr.shape[1]):
                new_arr[ridx, cidx] = arr[ridx][cidx].flatten()[0]
        return new_arr
    
    df_list = []
    for idx in book_idx_list:
        df_list += [pd.DataFrame(helper_tabular_option(arr[idx]), columns=['code', 'name', 'strike', 'date', 'exp_date'])]
    
    options_book = pd.concat(df_list, axis=0)
    options_book['exp_date'] = [str(date).replace('/', '-') for date in options_book['exp_date']]
    options_book['date'] = [str(date).replace('/', '-') for date in options_book['date']]
    options_book.insert(0, 'asof', [date.replace('/', '-') for i in range(options_book.shape[0])])
    options_book.insert(options_book.shape[1], 'time_to_exp', pd.to_datetime(options_book['exp_date']) - pd.to_datetime(options_book['asof']))
    
    # clean 
    options_book.drop_duplicates(inplace=True)
    
    # 
    type_list = [None] * options_book.shape[0]
    for idx, item in enumerate(options_book['name']):
        if item[5] == '购':
            type_list[idx] = 'CALL'
        else:
            type_list[idx] = 'PUT'
    options_book.insert(3, 'type', type_list)
    
    return options_book

In [256]:
def get_full_options_data(arr: np.array, dates: list):
    book_list = [get_daily_options_book(arr[idx], dates[idx]) for idx in range(len(dates))]
    
    options_book = pd.concat(book_list, axis=0)
    
    return options_book

In [257]:
full = get_full_options_data(C, dates)

## Strangle

In [358]:
from src.strategy import get_daily_strangle

In [366]:
%%time
security_selected = []
for date in full['asof'].unique():
    daily = full[full['asof'] == date].reset_index(drop=True)
    security_selected += get_daily_strangle(
        date=date, 
        options_book=daily, 
        ETF=ETF
    )

CPU times: user 1min 23s, sys: 1.43 s, total: 1min 24s
Wall time: 1min 28s


In [367]:
pd.DataFrame(security_selected)

,asof,code,name,type,strike,date,exp_date,time_to_exp
0,2015-02-16,10000049.SH,50ETF购3月2500,CALL,2.5,2015-02-12,2015-03-25,37 days
1,2015-02-16,10000008.SH,50ETF沽3月2300,PUT,2.3,2015-02-09,2015-03-25,37 days
2,2015-02-16,10000051.SH,50ETF购4月2500,CALL,2.5,2015-02-12,2015-04-22,65 days
3,2015-02-16,10000018.SH,50ETF沽4月2300,PUT,2.3,2015-02-09,2015-04-22,65 days
4,2015-02-16,10000053.SH,50ETF购6月2500,CALL,2.5,2015-02-12,2015-06-24,128 days
...,...,...,...,...,...,...,...,...
12492,2021-06-11,10003426.SH,50ETF沽7月3400,PUT,3.4,2021-5-27,2021-7-28,47 days
12493,2021-06-11,10003208.SH,50ETF购9月3700,CALL,3.7,2021-1-28,2021-9-22,103 days
12494,2021-06-11,10003214.SH,50ETF沽9月3400,PUT,3.4,2021-1-28,2021-9-22,103 days
12495,2021-06-11,10003379.SH,50ETF购12月3700,CALL,3.7,2021-4-29,2021-12-22,194 days


***MISSNG OPTIONS PRICE DATA***

# ETF 

In [112]:
ETF = mat['Contract'].flatten()[0][1]
ETF = pd.DataFrame(
    ETF, 
    index=pd.to_datetime(np.array(date_list)), 
    columns=['open', 'high', 'low', 'close', 'adj_close']
)
ETF

,open,high,low,close,adj_close
2015-02-16,2.390,2.403,2.379,2.394,2.3900
2015-02-17,2.401,2.429,2.401,2.411,2.4155
2015-02-25,2.411,2.411,2.359,2.370,2.3850
2015-02-26,2.368,2.452,2.348,2.450,2.4051
2015-02-27,2.447,2.465,2.428,2.438,2.4494
...,...,...,...,...,...
2021-06-07,3.604,3.606,3.570,3.593,3.5840
2021-06-08,3.591,3.630,3.550,3.572,3.5850
2021-06-09,3.570,3.590,3.562,3.582,3.5780
2021-06-10,3.582,3.630,3.577,3.601,3.6090


# IH

In [219]:
IH = mat['Contract'].flatten()[0][2]
print(IH.shape)

(1531, 14)
